In [ ]:
import firedrake
from firedrake import Constant, inner, grad, div, dx, exp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def solve_diffusion(Q, k, f, q_Γ):
    q = firedrake.Function(Q)
    ϕ = firedrake.TestFunction(Q)
    F = (k * inner(grad(q), grad(ϕ)) - f * ϕ) * dx
    bc = firedrake.DirichletBC(Q, q_Γ, "on_boundary")
    firedrake.solve(F == 0, q, bc)
    return q

In [ ]:
def exact_solution(x):
    zs = [
        Constant((1/4, 0)),
        Constant((-1/2, -1/4)),
        Constant((-1/2, +1/4)),
    ]
    rs = [Constant(1/4), Constant(1/8), Constant(1/8)]
    αs = [Constant(-1.0), Constant(1.0), Constant(1.0)]
    return sum([α * exp(-inner(x - z, x - z) / r**2) for z, r, α in zip(zs, rs, αs)])

In [ ]:
def step(mesh, degree):
    cell_size = mesh.cell_sizes.dat.data_ro[:].min()
    element = firedrake.FiniteElement("CG", "triangle", degree)
    Q = firedrake.FunctionSpace(mesh, element)
    x = firedrake.SpatialCoordinate(mesh)
    q_exact = exact_solution(x)
    f = -div(grad(q_exact))
    q_Γ = q_exact
    q_numeric = solve_diffusion(Q, Constant(1.0), f, q_Γ)
    error = firedrake.norm(q_exact - q_numeric, "H1")
    return cell_size, error

In [ ]:
nmin, nmax = 3, 9
data1 = np.array([step(firedrake.UnitDiskMesh(n), 1) for n in range(nmin, nmax)])
cell_sizes1, errors1 = data1[:, 0], data1[:, 1]

nmin, nmax = 3, 8
data2 = np.array([step(firedrake.UnitDiskMesh(n), 2) for n in range(nmin, nmax)])
cell_sizes2, errors2 = data2[:, 0], data2[:, 1]

In [ ]:
fig, ax = plt.subplots()
ax.set_xscale("log")
ax.set_yscale("log")
ax.scatter(cell_sizes1, errors1, color="tab:blue")
ax.plot(cell_sizes1, errors1, color="tab:blue")
ax.scatter(cell_sizes2, errors2, color="tab:orange")
ax.plot(cell_sizes2, errors2, color="tab:orange");

In [ ]:
slope1, intercept1 = np.polyfit(np.log(cell_sizes1), np.log(errors1), 1)
print(f"error (degree 1) ~= {intercept1:.2f} * δx^{slope1:.2f}")
slope2, intercept2 = np.polyfit(np.log(cell_sizes2), np.log(errors2), 1)
print(f"error (degree 2) ~= {intercept2:.2f} * δx^{slope2:.2f}")